# Ceneo scraper

## Struktura opinii w Ceneo
|Składowa|selektor|zmianna|
|--------|--------|-------|
|identyfikator opinii|["data-entry-id"]|opinion_id|
|autora|span.user-post__author-name|author|
|rekomendację|span.user-post__author-recomendation > em|recommend|
|liczbę gwiazdek|span.user-post__score-count|stars|
|data wystawienia opinii|span.user-post__published > time:nth-child(1)["datetime"]|opinion_date|
|data zakupu produktu|span.user-post__published > time:nth-child(2)["datetime"]|purchased_date|
|ile osób uznało opinię za przydatną|button.vote-yes["data-total-vote"] |up_vote|
|ile osób uznało opinię za nieprzydatną|button.vote-no["data-total-vote"]|down_vote|
|treść opinii|div.user-post__text|content|
|listę wad|review-feature__title--negatives ~ div.review-feature__item|cons|
|listę zalet|review-feature__title--positives ~ div.review-feature__item|pros|


In [7]:
import requests
import os
import json
from bs4 import BeautifulSoup

In [6]:

selectors = {

"author" :  ("span.user-post__author-name"),
"recommend" : ( "span.user-post__author-recomendation > em"),
"stars" : ( "span.user-post__score-count"),
"opinion_date" : (("span.user-post__published > time:nth-child(1)"),"datetime"),
"purchased_date" : (("span.user-post__published > time:nth-child(2)"),"datetime"),
"up_vote" : (("button.vote-yes"), "data-total-vote"),
"down_vote" : ( ("button.vote-no"), "data-total-vote"),
"content" : ( ("div.user-post__text")),
"cons" : ( ("div.review-feature__title--negatives ~ div.review-feature__item"), None, True),
"pros" : ( ("div.review-feature__title--positives ~ div.review-feature__item"), None, True),
"opinion_id" : ( None,"data-entry-id")

}


{'author': 't...k', 'recommend': 'Polecam', 'stars': '5/5', 'opinion_date': '2021-07-19 18:52:28', 'purchased_date': '2021-06-24 20:49:17', 'up_vote': '1', 'down_vote': '1', 'content': 'Odkurzacz lekki i cichy, ssanie mogło być mocniejsze, ale w tej cenie jest OK.', 'cons': [], 'pros': ['lekki', 'stosunkowo cichy', 'tani'], 'opinion_id': '14738443'}


In [5]:
def extract(ancestor, selector=None, attribute=None, return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.text.strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()


In [8]:
product_id = 106579657
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

In [9]:
response = requests.get(url)
response.status_code
response.text

'\r\n\r\n<!DOCTYPE html>\r\n<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7  non-direct ab-inactive" lang="pl"> <![endif]-->\r\n<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8  non-direct ab-inactive" lang="pl"> <![endif]-->\r\n<!--[if IE 8]>         <html class="no-js lt-ie9  non-direct ab-inactive" lang="pl"> <![endif]-->\r\n<!--[if gt IE 8]><!-->\r\n<html class="no-js  non-direct ab-inactive" lang="pl">\r\n<!--<![endif]-->\r\n<head>\r\n    <meta charset="utf-8">\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n    <link rel="preconnect" href="https://image.ceneo.pl">\n<link rel="preconnect" href="https://image.ceneostatic.pl">\n<link rel="preconnect" href="https://www.google-analytics.com">\n<link rel="preconnect" href="https://www.googleadservices.com">\n<link rel="preconnect" href="https://adservice.google.com">\n<link rel="preconnect" href="https://www.google.pl">\n<link rel="preconnect" href="https://www.google.com">\n<link rel="preconnect" 

In [10]:
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review")
opinion = page.select_one("div.js_product-review")
print(opinions)

[<div class="user-post user-post__card js_product-review" data-entry-id="14738443">
<header class="user-post__header">
<div class="js_lazy user-post__avatar user-rank__avatar" data-bg="/Content/img/account/avatar/9.svg"></div>
</header>
<div class="user-post__body">
<div class="user-post__content">
<span class="user-post__author-name">
t...k</span>
<span class="user-post__author-recomendation">
<em class="recommended">Polecam</em>
</span>
<span class="user-post__score">
<span class="screen-reader-text">Ocena:</span>
<span class="score-container score-container--s js_score-container">
<span class="score-marker score-marker--s" style="width: 100.00%;"></span>
</span>
<span class="user-post__score-count">5/5</span>
<span class="user-post__published">
        Wystawiono
        <time datetime="2021-07-19 18:52:28">3 lata temu, </time>
<time datetime="2021-06-24 20:49:17">po miesiącu</time> użytkowania            </span>
</span>
<div class="user-post__text">Odkurzacz lekki i cichy, ssanie m

In [ ]:
all_opinions = []
for opinion in opinions:
    single_opinion = { 
        key: extract(opinion, *value)
            for key, value in selectors.items()
}
    all_opinions.append(single_opinion)

In [ ]:
if not os.path.exists('opinions'):
    os.mkdir("opinions")
with open(f'opinions{product_id}.json', "w",encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=8, ensure_ascii=False)